# Reproduction

## Set-up

Import required packages

In [1]:
# Import model
import model

# Data processing and figure generation
import pandas as pd

# Manage file paths
from dataclasses import dataclass
import os

# Hide SettingWithCopyWarning error
# TODO: Resolve error
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Import and start timer (used to record runtime of this notebook)
import time
start = time.time()

Set file paths

In [2]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    outputs = '../outputs'
    endday7 = 'endday7.csv'
    endday14 = 'endday14.csv'
    endday21 = 'endday21.csv'


paths = Paths()

## Run model

In [3]:
EndDay7, EndDay14, EndDay21 = model.run_scenarios()

/home/amy/mambaforge/envs/lim2020/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/amy/mambaforge/envs/lim2020/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/amy/mambaforge/envs/lim2020/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/amy/mambaforge/envs/lim2020/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/amy/mambaforge/envs/lim2020/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(

In [4]:
display(EndDay7)
display(EndDay14)
display(EndDay21)

,2-1,2-3,2-7,2-14,2-21,4-1,4-3,4-7,4-14,4-21,6-1,6-3,6-7,6-14,6-21
5,NaN,NaN,NaN,NaN,NaN,0.10,0.10,0.15,0.15,0.20,0.07,0.07,0.13,0.10,0.13
10,NaN,NaN,NaN,NaN,NaN,0.07,0.10,0.20,0.23,0.20,0.03,0.07,0.15,0.13,0.13
20,NaN,NaN,NaN,NaN,NaN,0.11,0.16,0.25,0.25,0.25,0.04,0.09,0.17,0.17,0.17
30,NaN,NaN,NaN,NaN,NaN,0.18,0.20,0.25,0.25,0.25,0.06,0.13,0.17,0.17,0.17


,2-1,2-3,2-7,2-14,2-21,4-1,4-3,4-7,4-14,4-21,6-1,6-3,6-7,6-14,6-21
5,NaN,NaN,NaN,NaN,NaN,0.15,0.15,0.15,0.25,0.25,0.07,0.10,0.13,0.17,0.17
10,NaN,NaN,NaN,NaN,NaN,0.19,0.25,0.20,0.25,0.25,0.07,0.15,0.15,0.17,0.17
20,NaN,NaN,NaN,NaN,NaN,0.44,0.49,0.25,0.25,0.25,0.17,0.33,0.17,0.17,0.17
30,NaN,NaN,NaN,NaN,NaN,0.79,0.63,0.25,0.25,0.25,0.33,0.48,0.17,0.17,0.17


,2-1,2-3,2-7,2-14,2-21,4-1,4-3,4-7,4-14,4-21,6-1,6-3,6-7,6-14,6-21
5,NaN,NaN,NaN,NaN,NaN,0.25,0.30,0.30,0.25,0.25,0.10,0.13,0.17,0.17,0.17
10,NaN,NaN,NaN,NaN,NaN,0.42,0.45,0.42,0.25,0.25,0.13,0.27,0.28,0.17,0.17
20,NaN,NaN,NaN,NaN,NaN,0.90,0.79,0.49,0.25,0.25,0.48,0.61,0.36,0.17,0.17
30,NaN,NaN,NaN,NaN,NaN,0.99,0.93,0.49,0.25,0.25,0.79,0.78,0.39,0.17,0.17


## Save results to csv

In [5]:
EndDay7.to_csv(os.path.join(paths.outputs, paths.endday7), index=False)
EndDay14.to_csv(os.path.join(paths.outputs, paths.endday14), index=False)
EndDay21.to_csv(os.path.join(paths.outputs, paths.endday21), index=False)

## Time elapsed

In [6]:
# Find run time in seconds
end = time.time()
runtime = round(end-start)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime//60}m {runtime%60}s')

Notebook run time: 2m 40s
